# A scanner to help me find XSS vulnerabilties
With the end goal being to find a my own XSS vulnerability in the wild.
### Step 1: 
- Directory walk a domain and the grab the HTML/Javascript of each page where there is user input.

### Step 2:
- Inject my own custom string into each payload (slowly)

### Step 3:
- Capture and save the HTML/Javascript response and use regex to find my custom string in the response.

### Step 4: 
- Depending on how and where the string is presented in the response might give away that it's vulnerable to XSS. 




In [5]:
import requests
from urllib.parse import urljoin
import os
import re

# Function to sanitize file names derived from URLs
def sanitize_filename(path):
    return path.strip("/").replace("/", "_") or "root"

# Function to save the page source to a local file
def save_page_source(url, content):
    # Use the URL path to create a meaningful filename
    filename = f"{sanitize_filename(url.replace('https://', ''))}.html"
    
    # Ensure the directory exists
    os.makedirs('Valid_URLs_source', exist_ok=True)
    full_path = os.path.join('Valid_URLs_source', filename)

    # Save the content to the file
    with open(full_path, 'w', encoding='utf-8') as file:
        file.write(content)
    print(f"Saved page source to {full_path}")

# Define the root domain
root_domain = 'https://facebook.com'

# Define a list of paths to check
paths = ['', 'about/', 'contact/', 'nonexistentpage/', 'search/']

# Function to check if a URL is valid
def is_valid_url(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code < 400
    except requests.RequestException:
        return False

valid_urls = []
# Iterate over the paths and check each one
for path in paths:
    # Construct the full URL
    url = urljoin(root_domain, path)

    # Check if the URL is valid
    if is_valid_url(url):
        print(f"Valid URL: {url}")
        valid_urls.append(url)  # Add the valid URL to the list
    else:
        print(f"Invalid URL: {url}")

print('---------------------------------------')
print('Saving source code of valid pages...')
print('---------------------------------------')

for valid_url in valid_urls:
    try:
        response = requests.get(valid_url)
        if response.status_code < 400:
            save_page_source(valid_url, response.text)
        else:
            print(f"Failed to fetch the page source for {valid_url}")
    except requests.RequestException as e:
        print(f"Error fetching page source for {valid_url}: {e}")

print('---------------------------------------')
print('All valid URLs source code saved....')
print('---------------------------------------')
print('---------------------------------------')
print('Searching source for input tags.....')
print('---------------------------------------')

# Define the path to the folder containing the files
folder_path = 'C:\\Users\\User\\Documents\\Python Scripts\\JXSS\\Valid_URLs_source'

# Define the list of strings you want to search for
search_strings = ['<input', '<form', '<textarea']

# Function to search for the list of strings in a file
def search_strings_in_file(file_name, strings):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        # Iterate over each line in the file
        for line_number, line in enumerate(file, 1):
            # Check each string in the list
            for string in strings:
                if string in line:
                    # Print the string, file name, line number, and the line where the string is found
                    print(f"'{string}' found in {file_name} on line {line_number}.")

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    # Make sure to skip directories
    if os.path.isfile(os.path.join(folder_path, filename)):
        search_strings_in_file(filename, search_strings)







Valid URL: https://facebook.com
Valid URL: https://facebook.com/about/
Valid URL: https://facebook.com/contact/
Valid URL: https://facebook.com/nonexistentpage/
Valid URL: https://facebook.com/search/
---------------------------------------
Saving source code of valid pages...
---------------------------------------
Saved page source to Valid_URLs_source\facebook.com.html
Saved page source to Valid_URLs_source\facebook.com_about.html
Saved page source to Valid_URLs_source\facebook.com_contact.html
Saved page source to Valid_URLs_source\facebook.com_nonexistentpage.html
Failed to fetch the page source for https://facebook.com/search/
---------------------------------------
All valid URLs source code saved....
---------------------------------------
---------------------------------------
Searching source for input tags.....
---------------------------------------
'<input' found in facebook.com.html on line 7.
'<form' found in facebook.com.html on line 7.


In [4]:
# pip install webdriver-manager
# pip install selenium

# from banner import banner

# print(banner())

pip install validators

SyntaxError: invalid syntax (2984190939.py, line 8)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# URL you want to navigate to
url = 'http://example.com'

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
# This option opens the DevTools window automatically for each tab.
chrome_options.add_argument("--auto-open-devtools-for-tabs")

# Set up the Chrome driver (make sure ChromeDriver is in your PATH or use WebDriverManager)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the URL
driver.get(url)

# The browser should open with the DevTools window already open for the tab


In [ ]:
# Path to file
file_path = 'C:\\Users\\User\\Desktop\\marquee.html'

# List of XXS vectors to search for
search_strings = [
    '><marquee><', # Unsanitized HTML
    '= <marquee>;', # Unsanitized DOM or Javascript Variable set with User Data
    '=<marquee>">', # Embedded URL
    '(<marquee>)"', # Inline event handler
    'this_is_a_test_string'
]

# Open HTML file and read line by line:
with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        for search_string in search_strings:
            if search_string in line:
                if search_string == '><marquee><':
                    print(f"Found possible unsanitized string in HTML: '{search_string}' on line {line_number}: {line.strip()}")

                elif search_string == '= <marquee>;':
                    print(f"Found possible unsanitized input in InnerHTML or Javascript variable set with string: '{search_string}' on line {line_number}: {line.strip()}")
                
                elif search_string == '=<marquee>">':
                    print(f"Found possible embedded URL: '{search_string}' on line {line_number}: {line.strip()}")
                
                elif search_string == '(<marquee>)"':
                    print(f"Found possible inline event handlers with string: '{search_string}' on line {line_number}: {line.strip()}")
                
        else:
            continue  # Continue to the next line if no match is found
